In [2]:
import requests
import pandas as pd
import time
import datetime
import json
import re
from bs4 import BeautifulSoup

In [30]:
df_cbb = pd.read_csv(r'C:\Users\RJ\OneDrive\education\mads\591_592\milestone\02_sport_rawdata\cbb_2010_2019.csv')


&nbsp;



# Selenium


In [63]:
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\RJ\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
C:\Users\RJ\AppData\Local\Temp/ipykernel_14440/2518127973.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [166]:
seasons = ['2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19']


&nbsp;



#### Obtain game string


#### Michigan

In [167]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# driver = webdriver.Chrome('C:/Program Files/chromedriver/chromedriver.exe')

s = Service('C:/Program Files/chromedriver/chromedriver.exe')
browser = webdriver.Chrome(service=s)

game_strings = []
for season in seasons:
    url = f'https://msuspartans.com/sports/mens-basketball/schedule/{season}'
    browser.get(url)
    time.sleep(2)
    browser.implicitly_wait(0.5)

    # open each game's flyout
#     elements = browser.find_elements(By.XPATH, "//button[@class='sidearm-schedule-game-toggle noprint']")
#     for e in elements:
#         browser.execute_script("arguments[0].click();", e)
#         browser.implicitly_wait(0.5)
#         time.sleep(1)
    
    # pull strings from anchor tags
    elements = browser.find_elements(By.TAG_NAME, 'a')
    for e in elements:
        if 'Story' in e.accessible_name or 'Notes' in e.accessible_name or 'Box score' in e.accessible_name or 'Boxscore' in e.accessible_name or 'Boxcsore' in e.accessible_name or 'Stats for' in e.accessible_name or 'Quotes' in e.accessible_name or "Photos for" in e.accessible_name:
            pass
        elif ' PM' in e.accessible_name or ' AM' in e.accessible_name or ' p.m.' in e.accessible_name or 'Noon' in e.accessible_name or ' noon' in e.accessible_name or '(Exhib.)' in e.accessible_name:
            game_strings.append(e.accessible_name)
#         game_strings.append(e.get_attribute('innerHTML'))
    browser.implicitly_wait(0.5)

In [168]:
game_strings

['Green and White Game on October 24 3:30 PM',
 'Grand Valley State on November 9 7:00 PM',
 'Florida Gulf Coast on November 13 7:00 PM',
 'Gonzaga on November 17 8:00 PM',
 'Toledo on November 20 6:30 PM',
 'Valparaiso on November 22 12:00 PM',
 'Florida on November 27 8:00 PM',
 'Massachusetts on November 28 5:30 PM',
 'North Carolina (Big Ten/ACC Challenge) on December 1 9:00 PM',
 'Oakland on December 10 7:00 PM',
 'IPFW on December 19 12:00 PM',
 'Texas on December 22 7:00 PM',
 'Northwestern on January 2 6:30 PM',
 'Wisconsin on January 6 6:30 PM',
 'Iowa on January 9 5:35 PM',
 'Minnesota on January 13 6:30 PM',
 'Illinois on January 16 3:30 PM',
 'Iowa on January 20 6:30 PM',
 'Minnesota on January 23 12:00 PM',
 'Michigan on January 26 7:00 PM',
 'Northwestern on January 30 7:00 PM',
 'Wisconsin on February 2 9:00 PM',
 'Illinois on February 6 9:00 PM',
 'Purdue on February 9 9:00 PM',
 'Penn State on February 13 12:00 PM',
 'Indiana on February 16 7:00 PM',
 'Ohio State on Fe

In [173]:
len(game_strings)

335

In [90]:
game_strings = pd.Series(game_strings, name='Game String')

#### Michigan State

In [174]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

driver = webdriver.Chrome('C:/Program Files/chromedriver/chromedriver.exe')

game_strings = []
for season in seasons:
    driver.get(f'https://msuspartans.com/sports/mens-basketball/schedule/{season}')
    time.sleep(2)
    driver.implicitly_wait(0.5)
    
    # pull strings from anchor tags
    elements = driver.find_elements(By.XPATH, "//div[@class='sidearm-schedule-game-opponent-name']")
    for e in elements:
#         if 'Story' in e.accessible_name or 'Notes' in e.accessible_name or 'Box score' in e.accessible_name or 'Boxscore' in e.accessible_name or 'Boxcsore' in e.accessible_name or 'Stats for' in e.accessible_name or 'Quotes' in e.accessible_name or "Photos for" in e.accessible_name:
#             pass
#         elif ' PM' in e.accessible_name or ' AM' in e.accessible_name or ' p.m.' in e.accessible_name or 'Noon' in e.accessible_name or ' noon' in e.accessible_name or '(Exhib.)' in e.accessible_name:
#             game_strings.append(e.accessible_name)
        game_strings.append(e.text)
    driver.implicitly_wait(0.5)

C:\Users\RJ\AppData\Local\Temp/ipykernel_9796/816620328.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Program Files/chromedriver/chromedriver.exe')


In [175]:
game_strings

['GREEN AND WHITE GAME',
 'NORTHWOOD UNIVERSITY (EXHIB.)',
 'GRAND VALLEY STATE',
 'FLORIDA GULF COAST',
 'GONZAGA',
 'TOLEDO',
 'VALPARAISO',
 'FLORIDA',
 'MASSACHUSETTS',
 'NORTH CAROLINA (BIG TEN/ACC CHALLENGE)',
 'WOFFORD',
 'THE CITADEL',
 'OAKLAND',
 'IPFW',
 'TEXAS',
 'TEXAS-ARLINGTON',
 'NORTHWESTERN',
 'WISCONSIN',
 'IOWA',
 'MINNESOTA',
 'ILLINOIS',
 'IOWA',
 'MINNESOTA',
 'MICHIGAN',
 'NORTHWESTERN',
 'WISCONSIN',
 'ILLINOIS',
 'PURDUE',
 'PENN STATE',
 'INDIANA',
 'OHIO STATE',
 'PURDUE',
 'PENN STATE',
 'MICHIGAN',
 'MINNESOTA',
 'NEW MEXICO STATE',
 'MARYLAND',
 'NORTHERN IOWA',
 'TENNESSEE',
 'BUTLER',
 'MIDNIGHT MADNESS',
 'SAGINAW VALLEY STATE',
 'NEBRASKA-OMAHA',
 'EASTERN MICHIGAN',
 'SOUTH CAROLINA',
 'CHAMINADE',
 'CONNECTICUT',
 'WASHINGTON',
 'TENNESSEE TECH',
 'DUKE (ACC/BIG TEN CHALLENGE)',
 'BOWLING GREEN',
 'SYRACUSE',
 'OAKLAND',
 'PRAIRIE VIEW A&M',
 'TEXAS',
 'MINNESOTA',
 'NORTHWESTERN',
 'PENN STATE',
 'WISCONSIN',
 'NORTHWESTERN',
 'ILLINOIS',
 'PURDUE'

In [176]:
len(game_strings)

390

In [177]:
game_strings = pd.Series(game_strings, name='Game String')


&nbsp;



#### Obtain game start times


In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
driver = webdriver.Chrome('C:/Program Files/chromedriver/chromedriver.exe')

season_times = []
for season in seasons:
    driver.get(f'https://msuspartans.com/sports/mens-basketball/schedule/{season}')
    time.sleep(2)
    driver.implicitly_wait(0.5)
    
    # pull times from anchor tags
    elements = driver.find_elements(By.TAG_NAME, 'a')
    for e in elements:
        if ' vs ' in e.accessible_name or ' at ' in e.accessible_name:
            pass
        elif ' PM' in e.accessible_name or ' p.m.' in e.accessible_name or ' AM' in e.accessible_name or ' ET' in e.accessible_name or ' Noon' in e.accessible_name or ' noon' in e.accessible_name:
            season_times.append(re.findall('\d+:\d+ \w+|(?<=\d )\d+ [\w\.]+|[Nn]oon', e.accessible_name)[0])
    driver.implicitly_wait(0.5)

C:\Users\RJ\AppData\Local\Temp/ipykernel_9796/3586138797.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Program Files/chromedriver/chromedriver.exe')


In [77]:
season_times

[]

In [87]:
len(season_times)

390

In [91]:
season_times = pd.Series(season_times, name='Start time')

In [32]:
print(len(df_cbb[(df_cbb['school'] == 'michigan')]))
print(len(df_cbb[(df_cbb['school'] == 'michigan-state')]))

361
365



#### Michigan State


In [84]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
driver = webdriver.Chrome('C:/Program Files/chromedriver/chromedriver.exe')

season_times = []
for season in seasons:
    driver.get(f'https://msuspartans.com/sports/mens-basketball/schedule/{season}')
    time.sleep(2)
    driver.implicitly_wait(0.5)
    
    # pull times from anchor tags
    elements = driver.find_elements(By.XPATH, "//div[@class='sidearm-schedule-game-opponent-date flex-item-1']/span/following-sibling::span")
    for e in elements:
#         if ' vs ' in e.accessible_name or ' at ' in e.accessible_name:
#             pass
#         elif ' PM' in e.accessible_name or ' p.m.' in e.accessible_name or ' AM' in e.accessible_name or ' ET' in e.accessible_name or ' Noon' in e.accessible_name or ' noon' in e.accessible_name:
#             season_times.append(re.findall('\d+:\d+ \w+|(?<=\d )\d+ [\w\.]+|[Nn]oon', e.accessible_name)[0])
        if e.text == '':
            pass
        else:
            season_times.append(e.text)
    driver.implicitly_wait(0.5)

C:\Users\RJ\AppData\Local\Temp/ipykernel_9796/131651656.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Program Files/chromedriver/chromedriver.exe')


In [85]:
season_times

['3:30 PM',
 '4:00 PM',
 '7:00 PM',
 '7:00 PM',
 '8:00 PM',
 '6:30 PM',
 '12:00 PM',
 '8:00 PM',
 '5:30 PM',
 '9:00 PM',
 '7:00 PM',
 '7:00 PM',
 '7:00 PM',
 '12:00 PM',
 '7:00 PM',
 '7:00 PM',
 '6:30 PM',
 '6:30 PM',
 '5:35 PM',
 '6:30 PM',
 '3:30 PM',
 '6:30 PM',
 '12:00 PM',
 '7:00 PM',
 '7:00 PM',
 '9:00 PM',
 '9:00 PM',
 '9:00 PM',
 '12:00 PM',
 '7:00 PM',
 '12:00 PM',
 '4:00 PM',
 '7:00 PM',
 '4:00 PM',
 '9:00 PM',
 '7:20 PM',
 '2:30 PM',
 '9:37 PM',
 '2:20 PM',
 '6:07 PM',
 '9:30 PM',
 '7:00 PM',
 '7:00 PM',
 '8:30 PM',
 '10:00 PM',
 '9:30 PM',
 '7:00 PM',
 '5:00 PM',
 '1:00 PM',
 '9:30 PM',
 '1:30 PM',
 '9:00 PM',
 '12:30 PM',
 '6:30 PM',
 '7:00 PM',
 '4:00 PM',
 '7:30 PM',
 '1:00 PM',
 '7:00 PM',
 '1:00 PM',
 '7:00 PM',
 '9:00 PM',
 '7:00 PM',
 '6:00 PM',
 '8:30 PM',
 '1:00 PM',
 '7:00 PM',
 '9:00 PM',
 '9:00 PM',
 '9:00 PM',
 '1:00 PM',
 '6:30 PM',
 '2:00 PM',
 '5:00 PM',
 '6:30 PM',
 '4:15 PM',
 '9:20 PM',
 '2:00 PM',
 '7:00 PM',
 '7:00 PM',
 '7:00 PM',
 '6:30 PM',
 '6:00 PM

In [86]:
len(season_times)

390

In [178]:
season_times = pd.Series(season_times, name='Start time')


&nbsp;



#### Obtain US state locations


In [40]:
%%time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

s = Service('C:/Program Files/chromedriver/chromedriver.exe')
browser = webdriver.Chrome(service=s)

states = []
for season in seasons:
    url = (f'https://msuspartans.com/sports/mens-basketball/schedule/{season}')
    browser.get(url)
    browser.implicitly_wait(0.5)
    
    # pull states out of class
    elements = browser.find_elements(By.CLASS_NAME, 'sidearm-schedule-game-location')
    for e in elements:
        if e.text == 'CRISLER CENTER':
            states.append('MICH.')
        else:
            states.append(re.findall('(?<=, ).+', e.text)[0])
    browser.implicitly_wait(0.5)

Wall time: 18.6 s


In [41]:
len(states)

390

In [42]:
states

['MICH.',
 'MICH.',
 'MICH.',
 'MICH.',
 'MICH.',
 'MICH.',
 'MICH.',
 'N.J.',
 'N.J.',
 'N.C.',
 'MICH.',
 'S.C.',
 'MICH.',
 'MICH.',
 'TEXAS',
 'MICH.',
 'ILL.',
 'MICH.',
 'IOWA',
 'MICH.',
 'MICH.',
 'MICH.',
 'MINN.',
 'MICH.',
 'MICH.',
 'WIS.',
 'ILL.',
 'MICH.',
 'PA.',
 'IND.',
 'MICH.',
 'IND.',
 'MICH.',
 'MICH.',
 'IND.',
 'WASH.',
 'WASH.',
 'MO.',
 'MO.',
 'IND.',
 'MICH.',
 'MICH.',
 'MICH.',
 'MICH.',
 'MICH.',
 "HAWAI'I",
 "HAWAI'I",
 "HAWAI'I",
 'MICH.',
 'N.C.',
 'MICH.',
 'N.Y.',
 'MICH.',
 'MICH.',
 'MICH.',
 'MICH.',
 'ILL.',
 'PA.',
 'MICH.',
 'MICH.',
 'ILL.',
 'IND.',
 'MICH.',
 'MICH.',
 'IOWA',
 'WIS.',
 'MICH.',
 'OHIO',
 'MICH.',
 'MINN.',
 'MICH.',
 'MICH.',
 'MICH.',
 'IND.',
 'IND.',
 'IND.',
 'FLA.',
 'MICH.',
 'MICH.',
 'CALIF.',
 'N.Y.',
 'MICH.',
 'MICH.',
 'MICH.',
 'MICH.',
 'MICH.',
 'MICH.',
 'MICH.',
 'WASH.',
 'MICH.',
 'MICH.',
 'MICH.',
 'MICH.',
 'NEB.',
 'WIS.',
 'MICH.',
 'ILL.',
 'MICH.',
 'MICH.',
 'MICH.',
 'ILL.',
 'MICH.',
 'MICH.',


In [179]:
states = pd.Series(states, name='US State')


&nbsp;



#### Obtain attendance


In [251]:
%%time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

s = Service('C:/Program Files/chromedriver/chromedriver.exe')
browser = webdriver.Chrome(service=s)

msu_url = 'https://msuspartans.com/sports/mens-basketball/schedule/2009-10'

seasons = ['2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19']
attendance = []

for season in seasons:
    url = (f'https://mgoblue.com/sports/mens-basketball/schedule/{season}')
    browser.get(url)
    browser.implicitly_wait(0.5)

    # open each game's flyout
    elements = browser.find_elements(By.XPATH, "//button[@class='sidearm-schedule-game-toggle noprint']")
    for e in elements:
        browser.execute_script("arguments[0].click();", e)
        browser.implicitly_wait(0.5)
        time.sleep(1)

    # grab each game's attendance from <dd> tag after expanding each game
    elements = browser.find_elements(By.XPATH, "//div[@class='sidearm-schedule-game-extra-leaders flex-item-1']//dd[@data-bind='text: attendance']")
    for e in elements:
        if e.text != '':
            attendance.append(e.get_attribute('textContent'))
            browser.implicitly_wait(0.5)
            time.sleep(0.2)
    browser.implicitly_wait(0.5)

Wall time: 8min 40s


In [261]:
# find_elements adding an extra blank for unfound element at the end of each season's page
# some exhibition games do not have an attendance value, so they are skipped
# looping through 6 times, each time, an extra blank at the end - 3 times, number is not recorded, because it is blank (EX games)
# blanks must be appended, just not the last blank

print(len(attendance)) # add extra blank for each EX team where no attendance was recorded (where name == 'EX' and attendance = blank)
attendance

372


['',
 '9730',
 '10523',
 '2853',
 '3660',
 '2225',
 '10718',
 '10073',
 '9227',
 '11163',
 '16300',
 '10163',
 '15034',
 '13751',
 '5799',
 '11851',
 '9632',
 '13536',
 '17230',
 '14123',
 '13751',
 '13541',
 '5127',
 '13501',
 '14625',
 '9485',
 '13751',
 '11357',
 '18862',
 '10561',
 '14759',
 '14833',
 '16207',
 '',
 '7632',
 '7852',
 '7157',
 '6273',
 '5271',
 '7237',
 '9559',
 '9016',
 '9634',
 '8740',
 '9738',
 '8758',
 '13751',
 '11771',
 '17230',
 '12476',
 '11994',
 '17168',
 '5192',
 '12378',
 '14797',
 '12978',
 '18809',
 '8302',
 '10198',
 '13751',
 '16273',
 '13835',
 '11023',
 '14625',
 '13751',
 '17975',
 '18377',
 '16829',
 '18329',
 '',
 '11029',
 '7901',
 '8026',
 '2400',
 '2400',
 '2400',
 '10564',
 '10845',
 '17118',
 '9005',
 '10497',
 '11298',
 '12721',
 '12721',
 '16020',
 '12721',
 '12605',
 '12366',
 '12721',
 '19050',
 '14533',
 '18809',
 '12721',
 '14797',
 '9533',
 '12721',
 '12721',
 '8117',
 '12721',
 '15244',
 '9564',
 '18848',
 '18451',
 '11625',
 '10524

In [270]:
# insert empty values for the three exhibition games Michigan played w/out <dd> tags
attendance.insert(0, '') # Wayne State EX game with no attendance recorded
attendance.insert(33, '') # EX game with no attendance recorded
attendance.insert(69, '') # EX game with no attendance recorded
attendance.insert(341, '') # George Washington game had no attendance recorded

In [271]:
attendance_series = pd.Series(attendance, name='Attendance')


&nbsp;



# CBB DataFrame


In [102]:
df_cbb.dtypes

Unnamed: 0      int64
school         object
gamedate       object
type           object
opponent       object
conf           object
result         object
team_points     int64
opp_points      int64
ot             object
w               int64
l               int64
streak         object
arena          object
rank           object
gametime       object
dtype: object

In [105]:
df_cbb['gamedate'] = pd.to_datetime(df_cbb['gamedate'])

In [139]:
df_cbb[(df_cbb['school'] == ('michigan-state')) 
        & (df_cbb['gamedate'] >= ('2015-10-01')) 
        & (df_cbb['gamedate'] <= ('2016-05-01'))]

,Unnamed: 0,school,gamedate,type,opponent,conf,result,team_points,opp_points,ot,w,l,streak,arena,rank,gametime
582,612,michigan-state,2015-11-13,REG,Florida Atlantic,CUSA,W,82,55,NaN,1,0,W 1,Breslin Events Center,13,7:00p
583,613,michigan-state,2015-11-17,REG,Kansas (4),Big 12,W,79,73,NaN,2,0,W 2,United Center,13,10:00p
584,614,michigan-state,2015-11-20,REG,Arkansas-Pine Bluff,SWAC,W,92,46,NaN,3,0,W 3,Breslin Events Center,13,7:00p
585,615,michigan-state,2015-11-23,REG,Eastern Michigan,MAC,W,89,65,NaN,4,0,W 4,Breslin Events Center,3,7:00p
586,616,michigan-state,2015-11-26,REG,Boston College,ACC,W,99,68,NaN,5,0,W 5,Titan Gym,3,6:30p
587,617,michigan-state,2015-11-27,REG,Boise State,MWC,W,77,67,NaN,6,0,W 6,Titan Gym,3,5:30p
588,618,michigan-state,2015-11-29,REG,Providence,Big East,W,77,64,NaN,7,0,W 7,Honda Center,3,10:00p
589,619,michigan-state,2015-12-02,REG,Louisville (24),ACC,W,71,67,NaN,8,0,W 8,Breslin Events Center,3,7:15p
590,620,michigan-state,2015-12-05,REG,Binghamton,AEC,W,76,33,NaN,9,0,W 9,Breslin Events Center,3,12:00p
591,621,michigan-state,2015-12-09,REG,Maryland-Eastern Shore,MEAC,W,78,35,NaN,10,0,W 10,Breslin Events Center,1,7:00p



Michigan's game times differ from those of sports-reference.com in that Michigan applies timezone adjustment.


In [114]:
df_cbb[df_cbb['school'] == 'michigan-state'].head()

,Unnamed: 0,school,gamedate,type,opponent,conf,result,team_points,opp_points,ot,w,l,streak,arena,rank,gametime
361,379,michigan-state,2009-11-13,REG,Florida Gulf Coast,A-Sun,W,97,58,NaN,1,0,W 1,Breslin Events Center,2,NaN
362,380,michigan-state,2009-11-17,REG,Gonzaga,WCC,W,75,71,NaN,2,0,W 2,Breslin Events Center,2,NaN
363,381,michigan-state,2009-11-20,REG,Toledo,MAC,W,75,62,NaN,3,0,W 3,Breslin Events Center,2,NaN
364,382,michigan-state,2009-11-22,REG,Valparaiso,Horizon,W,90,60,NaN,4,0,W 4,Breslin Events Center,2,NaN
365,383,michigan-state,2009-11-27,REG,Florida,SEC,L,74,77,NaN,4,1,L 1,Boardwalk Hall,2,NaN


In [113]:
df_cbb[df_cbb['school'] == 'michigan-state'].tail()

,Unnamed: 0,school,gamedate,type,opponent,conf,result,team_points,opp_points,ot,w,l,streak,arena,rank,gametime
721,759,michigan-state,2019-03-21,NCAA,Bradley,MVC,W,76,65,NaN,29,6,W 6,Wells Fargo Arena,NaN,2:45p
722,760,michigan-state,2019-03-23,NCAA,Minnesota,Big Ten,W,70,50,NaN,30,6,W 7,Wells Fargo Arena,NaN,7:45p
723,761,michigan-state,2019-03-29,NCAA,Louisiana State (12),SEC,W,80,63,NaN,31,6,W 8,Capital One Arena,NaN,7:09p
724,762,michigan-state,2019-03-31,NCAA,Duke (1),ACC,W,68,67,NaN,32,6,W 9,Capital One Arena,NaN,5:05p
725,763,michigan-state,2019-04-06,NCAA,Texas Tech (9),Big 12,L,51,61,NaN,32,7,L 1,U.S. Bank Stadium,NaN,8:49p



&nbsp;



# Selenium DataFrame


In [257]:
df_selenium = pd.concat([game_strings, season_times, states], axis=1)
df_selenium

,Game String,Start time,US State
0,GREEN AND WHITE GAME,3:30 PM,MICH.
1,NORTHWOOD UNIVERSITY (EXHIB.),4:00 PM,MICH.
2,GRAND VALLEY STATE,7:00 PM,MICH.
3,FLORIDA GULF COAST,7:00 PM,MICH.
4,GONZAGA,8:00 PM,MICH.
...,...,...,...
385,BRADLEY,2:45 PM ET,IOWA / WELLS FARGO ARENA
386,MINNESOTA,7:45 PM ET,IOWA / WELLS FARGO ARENA
387,#12 LSU,7 PM ET,D.C. / CAPITAL ONE ARENA
388,#1 DUKE,5:05 PM ET,D.C. / CAPITAL ONE ARENA


In [258]:
# remove rows where '(EX)' in 'Game String' before pd.concat w/ df_cbb
df_selenium = df_selenium[~df_selenium['Game String'].str.contains('EXHIBITION|\(EX\)|EXHIB|MIDNIGHT|GREEN AND WHITE')]

In [259]:
df_selenium = df_selenium.reset_index(drop=True)

In [260]:
# drop exhibition games by row number (improve)
df_selenium = df_selenium.drop([0, 38, 39, 97, 121, 122, 124, 151, 152, 191, 192, 232, 233, 269, 270, 306, 307, 308, 344])

In [192]:
df_msu = df_cbb[df_cbb['school'] == 'michigan-state']
df_msu.to_csv(r'C:\Users\RJ\Downloads\df_msu.csv')

In [261]:
df_selenium = df_selenium.reset_index(drop=True)
df_selenium

,Game String,Start time,US State
0,FLORIDA GULF COAST,7:00 PM,MICH.
1,GONZAGA,8:00 PM,MICH.
2,TOLEDO,6:30 PM,MICH.
3,VALPARAISO,12:00 PM,MICH.
4,FLORIDA,8:00 PM,N.J.
...,...,...,...
360,BRADLEY,2:45 PM ET,IOWA / WELLS FARGO ARENA
361,MINNESOTA,7:45 PM ET,IOWA / WELLS FARGO ARENA
362,#12 LSU,7 PM ET,D.C. / CAPITAL ONE ARENA
363,#1 DUKE,5:05 PM ET,D.C. / CAPITAL ONE ARENA


In [228]:
df_selenium.to_csv(r'C:\Users\RJ\Downloads\selenium_basketball_data.csv', sep=',')

In [243]:
len(df_cbb[df_cbb['school'] == 'michigan-state'])

365


&nbsp;



# Final College Basketball DataFrame


In [ ]:
#split Michigan and Michigan State into separate DataFrames, concat columns, then concat vertically

In [352]:
df_m = df_cbb[df_cbb['school'] == 'michigan']
df_msu = df_cbb[df_cbb['school'] == 'michigan-state']

In [353]:
df_msu = df_msu.reset_index()

In [354]:
df_msu = pd.concat([df_msu, df_selenium], axis=1)

In [ ]:
# reimport df_msu from file
df_msu = pd.read_csv(r'../04_finaldata/bball_with_michigan_state.csv')

In [355]:
df_msu['Start time'] = df_msu['Start time'].replace('Noon', '12:00 PM')

In [371]:
df_msu['Start time'][325:326] = pd.to_datetime(df_msu['Start time'][325:326])

C:\Users\RJ\AppData\Local\Temp/ipykernel_9796/3380159653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_msu['Start time'][325:326] = pd.to_datetime(df_msu['Start time'][325:326])


In [375]:
df_msu['Start time'][326]

'7 P.M.'

In [378]:
df_msu['Start time'][326] = pd.to_datetime(df_msu['Start time'][326])

C:\ProgramData\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname M identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1-01-01 19:00:00

In [272]:
df_msu.to_csv(r'C:\Users\RJ\Downloads\bball_with_michigan_state.csv')

In [316]:
# reimport df w/ Michigan bball data scraped w/ Selenium
df_m = pd.read_csv(r'../04_finaldata/bball_data_with_michigans_numbers.csv', usecols=range(2, 20))

In [317]:
df_m['Start time'] = df_m['Start time'].replace('Noon', '12:00 PM')

In [318]:
df_m['Start time'] = pd.to_datetime(df_m['Start time'])

In [319]:
# Michigan's mgoblue gametimes all map to location's local time. Use dict to move each time back to EST.
time_dict = {'ILL.': 1, 'WIS.': 1, 'UTAH': 2, 'KAN.': 1, 'MINN.': 1, 'IOWA': 1, 'HAWAII': 6, 'ARK.': 1,
             'NEB.': 1, 'TEXAS': 1, 'ARIZ.': 2, 'CALIF.': 3, 'MO.': 1, 'HAWAI\'I': 1}

In [339]:
import datetime

def time_converter(dt, state):
    if state not in time_dict:
        return dt
    else:
        return dt + datetime.timedelta(hours=time_dict[state])

In [340]:
df_m['Start time'] = df_m.apply(lambda x: time_converter(x['Start time'], x['US State']), axis=1)

In [343]:
df_m[-40:-1]

,school,gamedate,type,opponent,conf,result,team_points,opp_points,ot,w,l,streak,arena,rank,gametime,Game String,Start time,US State
321,michigan,3/24/2018,NCAA,Florida State,ACC,W,58,54,NaN,32,7,W 13,Staples Center,NaN,8:49p,Florida State on March 24 5:49 PM PT,2022-05-10 20:49:00,CALIF.
322,michigan,3/31/2018,NCAA,Loyola (IL),MVC,W,69,57,NaN,33,7,W 14,Alamodome,NaN,6:09p,Loyola Chicago on March 31 5:09 PM CT,2022-05-10 18:09:00,TEXAS
323,michigan,4/2/2018,NCAA,Villanova (2),Big East,L,62,79,NaN,33,8,L 1,Alamodome,NaN,9:20p,Villanova on April 2 8:19 PM CT,2022-05-10 21:19:00,TEXAS
324,michigan,11/6/2018,REG,Norfolk State,MEAC,W,63,44,NaN,1,0,W 1,Crisler Arena,19,8:30p,Norfolk State on November 6 8:30 PM,2022-05-10 20:30:00,MICH.
325,michigan,11/10/2018,REG,Holy Cross,Patriot,W,56,37,NaN,2,0,W 2,Crisler Arena,19,7:30p,Holy Cross on November 10 7:30 PM,2022-05-10 19:30:00,MICH.
326,michigan,11/14/2018,REG,Villanova (8),Big East,W,73,46,NaN,3,0,W 3,Finneran Pavilion,18,6:30p,Villanova on November 14 6:30 p.m.,2022-05-10 18:30:00,PA.
327,michigan,11/17/2018,REG,George Washington,A-10,W,84,61,NaN,4,0,W 4,Mohegan Sun Arena,18,12:00p,George Washington on November 17 Noon,2022-05-10 12:00:00,CONN.
328,michigan,11/18/2018,REG,Providence,Big East,W,66,47,NaN,5,0,W 5,Mohegan Sun Arena,18,1:30p,Providence on November 18 1:30 PM,2022-05-10 13:30:00,CONN.
329,michigan,11/23/2018,REG,Chattanooga,Southern,W,83,55,NaN,6,0,W 6,Crisler Arena,9,4:00p,Chattanooga on November 23 4:00 PM,2022-05-10 16:00:00,MICH.
330,michigan,11/28/2018,REG,North Carolina (11),ACC,W,84,67,NaN,7,0,W 7,Crisler Arena,7,9:30p,North Carolina on November 28 9:30 PM,2022-05-10 21:30:00,MICH.


In [344]:
df_basketball = pd.concat([df_m, df_msu]).reset_index(drop=True)

In [345]:
df_basketball['gamedate'] = pd.to_datetime(df_basketball['gamedate'])

In [346]:
df_basketball[(df_basketball['school'] == ('michigan')) 
              & (df_basketball['gamedate'] >= ('2014-10-01')) 
              & (df_basketball['gamedate'] <= ('2015-05-01'))]

,school,gamedate,type,opponent,conf,result,team_points,opp_points,ot,w,l,streak,arena,rank,gametime,Game String,Start time,US State,index,Unnamed: 0
177,michigan,2014-11-15,REG,Hillsdale,NaN,W,92,68,NaN,1,0,W 1,Crisler Arena,24,2:00p,Hillsdale College on November 15 2:00 PM,2022-05-10 14:00:00,MICH.,NaN,NaN
178,michigan,2014-11-17,REG,Bucknell,Patriot,W,77,53,NaN,2,0,W 2,Crisler Arena,24,8:00p,Bucknell on November 17 8:00 PM,2022-05-10 20:00:00,MICH.,NaN,NaN
179,michigan,2014-11-20,REG,Detroit Mercy,Horizon,W,71,62,NaN,3,0,W 3,Crisler Arena,24,6:00p,Detroit on November 20 6:00 PM,2022-05-10 18:00:00,MICH.,NaN,NaN
180,michigan,2014-11-24,REG,Oregon,Pac-12,W,70,63,NaN,4,0,W 4,Barclays Center,19,9:45p,Oregon on November 24 9:00 PM,2022-05-10 21:00:00,N.Y.,NaN,NaN
181,michigan,2014-11-25,REG,Villanova (12),Big East,L,55,60,NaN,4,1,L 1,Barclays Center,19,10:15p,Villanova on November 25 10:00 PM,2022-05-10 22:00:00,N.Y.,NaN,NaN
182,michigan,2014-11-29,REG,Nicholls State,Southland,W,91,62,NaN,5,1,W 1,Crisler Arena,19,4:00p,Nicholls State on November 29 4:00 PM,2022-05-10 16:00:00,MICH.,NaN,NaN
183,michigan,2014-12-02,REG,Syracuse,ACC,W,68,65,NaN,6,1,W 2,Crisler Arena,17,7:30p,Syracuse on December 2 7:30 PM,2022-05-10 19:30:00,MICH.,NaN,NaN
184,michigan,2014-12-06,REG,NJIT,Ind,L,70,72,NaN,6,2,L 1,Crisler Arena,17,12:00p,NJIT on December 6 12:00 PM,2022-05-10 12:00:00,MICH.,NaN,NaN
185,michigan,2014-12-09,REG,Eastern Michigan,MAC,L,42,45,NaN,6,3,L 2,Crisler Arena,-,9:00p,Eastern Michigan on December 9 9:00 PM,2022-05-10 21:00:00,MICH.,NaN,NaN
186,michigan,2014-12-13,REG,Arizona (3),Pac-12,L,53,80,NaN,6,4,L 3,McKale Center,-,5:15p,Arizona on December 13 3:25 PM,2022-05-10 17:25:00,ARIZ.,NaN,NaN


In [347]:
df_basketball[-40:-1]

,school,gamedate,type,opponent,conf,result,team_points,opp_points,ot,w,l,streak,arena,rank,gametime,Game String,Start time,US State,index,Unnamed: 0
686,michigan-state,2018-03-18,NCAA,Syracuse,ACC,L,53,55,NaN,30,5,L 1,Little Caesars Arena,NaN,2:40p,SYRACUSE,2:40 PM,MICH. (LITTLE CAESARS ARENA),686.0,722.0
687,michigan-state,2018-11-06,REG,Kansas (1),Big 12,L,87,92,NaN,0,1,L 1,Bankers Life Fieldhouse,10,7:00p,#1 KANSAS,7 P.M.,IND.,687.0,723.0
688,michigan-state,2018-11-11,REG,Florida Gulf Coast,A-Sun,W,106,82,NaN,1,1,W 1,Breslin Events Center,10,6:00p,FLORIDA GULF COAST,6 P.M.,MICH. BRESLIN CENTER,688.0,724.0
689,michigan-state,2018-11-14,REG,Louisiana-Monroe,Sun Belt,W,80,59,NaN,2,1,W 2,Breslin Events Center,11,7:00p,LOUISIANA MONROE,7 P.M.,MICH. BRESLIN CENTER,689.0,725.0
690,michigan-state,2018-11-18,REG,Tennessee Tech,OVC,W,101,33,NaN,3,1,W 3,Breslin Events Center,11,6:00p,TENNESSEE TECH,6 P.M.,MICH. BRESLIN CENTER,690.0,726.0
691,michigan-state,2018-11-22,REG,UCLA (17),Pac-12,W,87,67,NaN,4,1,W 4,Orleans Arena,11,10:00p,#17 UCLA,10 PM ET,NEV.,691.0,727.0
692,michigan-state,2018-11-23,REG,Texas,Big 12,W,78,68,NaN,5,1,W 5,Orleans Arena,11,6:30p,TEXAS,6:30 PM ET,NEV.,692.0,728.0
693,michigan-state,2018-11-27,REG,Louisville,ACC,L,78,82,OT,5,2,L 1,KFC Yum! Center,9,7:30p,LOUISVILLE,7:30 PM ET,KY.,693.0,729.0
694,michigan-state,2018-11-30,REG,Rutgers,Big Ten,W,78,67,NaN,6,2,W 1,Louis Brown Athletic Center,9,6:00p,RUTGERS,6 PM ET,N.J.,694.0,730.0
695,michigan-state,2018-12-03,REG,Iowa (18),Big Ten,W,90,68,NaN,7,2,W 2,Breslin Events Center,10,6:30p,#18 IOWA,6:30 PM ET,MICH. BRESLIN CENTER,695.0,731.0


In [349]:
df_basketball.to_csv(r'C:\Users\RJ\Downloads\df_basketball.csv', sep=',')